In [1]:
!pip install openai --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


### Create JSON with UCL semi-final

In [35]:
import json

def get_ucl_semi_final_results(team):
  """Get the results of a given football game"""
  if "bayern" in team.lower() or "real" in team.lower():
    return json.dumps(
        {
            "team": "Bayern Munich", "score": "3", "stutus": "not qualified",
            "opponent": "Real Madrid", "opponent_score": "4", "opponent_stutus": "qualified"
        }
    )
  elif "paris" in team.lower() or "borussia" in team.lower():
    return json.dumps(
        {
            "team": "Paris Saint German", "score": "0", "stutus": "not qualified",
            "opponent": "Borussia Dortmund", "opponent_score": "2", "opponent_stutus": "qualified"
        }
    )
  else:
    return json.dumps(
        {
            "team": team,
            "score": "Does not exist",
            "stutus": "not qualified"
        }
    )

### Load the model

In [3]:
from openai import OpenAI
import os
from google.colab import userdata

model_name = "gpt-4o"

client = OpenAI(
    api_key = userdata.get("OPENAI_API_KEY")
)

In [21]:
def function_calling(prompt):
  messages = [
      {
          "role": "system",
          "content": """You are a function calling LLM that uses the data extracted from the get_ucl_semi_final_results function to answer questions around football game scores.
            Include the team and their opponent in your response, also include the status of each team"""
      },
      {
          "role": "user",
          "content": prompt
      }
  ]

  tools = [
      {
          "type": "function",
          "function": {
              "name": "get_ucl_semi_final_results",
              "description": "Get the score for a given Football game",
              "parameters": {
                  "type": "object",
                  "properties": {
                      "team": {
                          "type": "string",
                          "description": "The name of the Football team (e.g. 'Bayern Munich')",
                      },
                  },
              },
              "required": ["team"],
          }
      }
  ]

  response = client.chat.completions.create(
      model = model_name,
      messages = messages,
      tools = tools,
      tool_choice = 'auto'
  )

  response_message = response.choices[0].message
  tool_calls = response_message.tool_calls

  if tool_calls:
      available_functions = {
          "get_ucl_semi_final_results": get_ucl_semi_final_results,
      }

      messages.append(response_message)

      for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)

        print(f"Tool call: {tool_call}")

        function_response = function_to_call(
            team = function_args.get("team"),
        )

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

      second_response = client.chat.completions.create(
          model = model_name,
          messages = messages,
      )

      return second_response.choices[0].message.content

  else:
    return response_message

In [30]:
prompt = "What was the result Bayern Munich game?"
print(function_calling(prompt))

Tool call: ChatCompletionMessageToolCall(id='call_z4T1HcEJCpfyqtohRYNg61J4', function=Function(arguments='{"team":"Bayern Munich"}', name='get_ucl_semi_final_results'), type='function')
Bayern Munich played against Real Madrid and the result was 3-4. Bayern Munich did not qualify, while Real Madrid qualified.


In [33]:
prompt = "What was the result Borussia Dortmund game?"
print(function_calling(prompt))

Tool call: ChatCompletionMessageToolCall(id='call_kg4TfAWrn0yjtwUXM6R2EuvM', function=Function(arguments='{"team":"Borussia Dortmund"}', name='get_ucl_semi_final_results'), type='function')
Borussia Dortmund played against Paris Saint Germain and won with a score of 2-0. Borussia Dortmund qualified for the next round, while Paris Saint Germain did not qualify.


In [36]:
prompt = "What was the result Barcelona game?"
print(function_calling(prompt))

Tool call: ChatCompletionMessageToolCall(id='call_zGwbQUhVGYrriFY4MTiiUiCW', function=Function(arguments='{"team":"Barcelona"}', name='get_ucl_semi_final_results'), type='function')
Barcelona did not qualify for the UEFA Champions League semi-finals and therefore does not have any game results at this stage.


In [37]:
prompt = "What was the result Arsenal game?"
print(function_calling(prompt))

Tool call: ChatCompletionMessageToolCall(id='call_eJZzSyfjXtaT6mrGRqPaSxFq', function=Function(arguments='{\n  "team": "Arsenal"\n}', name='get_ucl_semi_final_results'), type='function')
Arsenal did not qualify for the semi-finals, so there is no result for an Arsenal game.
